In [317]:
# Imports

import open3d as o3d
import numpy as np
import cv2
import numpy as np 
from ultralytics import YOLO
import copy

In [318]:
# Data Path

model_data = r"C:\Hackathon\3D bone mapping\forearm_Bones.glb"
image_data = r"C:\Hackathon\3D bone mapping\working_code\test\test_5.png"

In [319]:
# Model and Image

mesh = o3d.io.read_triangle_mesh(model_data)
img = cv2.imread(image_data)

In [320]:
# Per-processing Model

mesh.remove_duplicated_vertices()
mesh.remove_degenerate_triangles()
mesh.remove_duplicated_triangles()
mesh.remove_non_manifold_edges()
mesh.remove_unreferenced_vertices()

mesh.scale(1 / np.max(mesh.get_max_bound() - mesh.get_min_bound()), center=mesh.get_center())
mesh.translate(-mesh.get_center())
mesh.compute_vertex_normals()

R1 = mesh.get_rotation_matrix_from_axis_angle([np.pi, 0, 0])
R2 = mesh.get_rotation_matrix_from_axis_angle([0, -np.pi / 2, 0])

R = R2 @ R1
mesh.rotate(R2, center=(0, 0, 0))

TriangleMesh with 485454 points and 970900 triangles.

In [321]:
# Bounding Box around model 

bbox = mesh.get_axis_aligned_bounding_box()
bbox.color = (1, 0, 0)

min_bound = bbox.get_min_bound()
max_bound = bbox.get_max_bound()
size = max_bound - min_bound

print(f"Width:  {size[0]:.4f}")
print(f"Height: {size[1]:.4f}")
print(f"Depth:  {size[2]:.4f}")
print("Model center:", mesh.get_center())

o3d.visualization.draw_geometries([mesh, bbox])

Width:  0.2318
Height: 1.0000
Depth:  0.1647
Model center: [ 3.4601e-14  1.0308e-12  6.0746e-13]


In [322]:
# Divide Mesh to make Clusters 

plane_height, plane_depth = size[1], size[2]
plane_thickness = 0.001

vertical_plane = o3d.geometry.TriangleMesh.create_box(
    width=plane_thickness,
    height=plane_height,
    depth=plane_depth
)

vertical_plane.translate((-plane_thickness/2, -plane_height/2, -plane_depth/2))

center_x = (min_bound[0] + max_bound[0]) / 2
vertical_plane.translate((center_x, 0, 0))

angle_from_x = 94.11222884471846
tilt_angle = angle_from_x - 90
angle_rad = np.deg2rad(-tilt_angle)

R = vertical_plane.get_rotation_matrix_from_axis_angle([0, 0, angle_rad])
vertical_plane.rotate(R, center=vertical_plane.get_center())

shift_amount = -0.015
vertical_plane.translate((shift_amount, 0, 0))

vertical_plane.paint_uniform_color([1, 0.7, 0.3])

o3d.visualization.draw_geometries([mesh, vertical_plane])


In [323]:
# Create Clusters

plane_normal = R @ np.array([1.0, 0.0, 0.0])
plane_normal /= np.linalg.norm(plane_normal)

plane_center = vertical_plane.get_center()
points = np.asarray(mesh.vertices)

signed_distances = np.dot(points - plane_center, plane_normal)

mask_above = signed_distances > 0
mask_below = signed_distances <= 0

mesh_ulna = mesh.select_by_index(np.where(mask_above)[0].tolist())
mesh_radius = mesh.select_by_index(np.where(mask_below)[0].tolist())

mesh_ulna.paint_uniform_color([0.2, 0.8, 1.0])
mesh_radius.paint_uniform_color([1.0, 0.4, 0.4])

o3d.visualization.draw_geometries([mesh_radius, mesh_ulna])

In [324]:
# Model Landmarks 

model_landmarks = {
    "ulna_head": (0.058, 0.35, 0.0),
    "ulna_tail": (0.0028, -0.43, 0.0),
    "radius_head": (-0.013, 0.35, 0.0),
    "radius_tail": (-0.07, -0.42, 0.0)
}

In [325]:
# Landmarks Marking Funcation

def draw_cylinder_between(p1, p2, radius=0.01, color=[1, 0, 0]):
    p1, p2 = np.array(p1), np.array(p2)
    axis = p2 - p1
    height = np.linalg.norm(axis)
    midpoint = (p1 + p2) / 2

    cylinder = o3d.geometry.TriangleMesh.create_cylinder(radius=radius, height=height)
    cylinder.paint_uniform_color(color)
    cylinder.compute_vertex_normals()

    z_axis = np.array([0, 0, 1])
    axis_norm = axis / np.linalg.norm(axis)
    v = np.cross(z_axis, axis_norm)
    c = np.dot(z_axis, axis_norm)

    if np.linalg.norm(v) < 1e-6:
        R = np.eye(3)
    else:
        vx = np.array([
            [0, -v[2], v[1]],
            [v[2], 0, -v[0]],
            [-v[1], v[0], 0]
        ])
        R = np.eye(3) + vx + vx @ vx * ((1 - c) / (np.linalg.norm(v) ** 2))

    cylinder.rotate(R, center=(0, 0, 0))
    cylinder.translate(midpoint)
    return cylinder

In [326]:
# Landmark Marking

ulna_line = draw_cylinder_between(model_landmarks["ulna_head"], model_landmarks["ulna_tail"], radius=0.01, color=[1, 0, 0])
radius_line = draw_cylinder_between(model_landmarks["radius_head"], model_landmarks["radius_tail"], radius=0.01, color=[0, 1, 0])

In [327]:
# Visualize Model with Landmarks 

o3d.visualization.draw_geometries([mesh, ulna_line, radius_line])

In [328]:
# Display Image 

cv2.imshow("X-ray", img)
cv2.waitKey(0)
cv2.destroyAllWindows() 

In [329]:
# Get Pixel Coordintes ( Manually )

clone = img.copy()

labels = ['ulna head', 'ulna tail', 'radius head', 'radius tail']
points = []
index = 0
Xray_landmark = {}

def click_landmarks(event, x, y, flags, param):
    global index, Xray_landmark

    if event == cv2.EVENT_LBUTTONDOWN and index < len(labels):
        label = labels[index]
        points.append((x, y))
        Xray_landmark[label] = (x, y)

        cv2.circle(clone, (x, y), 5, (0, 0, 255), -1)
        cv2.putText(clone, label, (x + 5, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
        index += 1
        cv2.imshow("Image", clone)

        if index == len(labels):
            h, w = clone.shape[:2]
            cx, cy = w // 2, h // 2

            for k in Xray_landmark:
                x, y = Xray_landmark[k]
                Xray_landmark[k] = (x - cx, cy - y)

            print("\n--> Centered Coordinates (origin at image center):")
            for key, val in Xray_landmark.items():
                print(f"{key} → {val}")

cv2.imshow("Image", clone)
cv2.setMouseCallback("Image", click_landmarks)
cv2.waitKey(0)
cv2.destroyAllWindows()


--> Centered Coordinates (origin at image center):
ulna head → (-8, 351)
ulna tail → (7, -431)
radius head → (-67, 287)
radius tail → (-58, -443)


In [330]:
labels = ['ulna break', 'radius break']
points = []
index = 0
Xray_breaks = {}

def click_landmarks(event, x, y, flags, param):
    global index, Xray_breaks

    if event == cv2.EVENT_LBUTTONDOWN and index < len(labels):
        label = labels[index]
        points.append((x, y))
        Xray_breaks[label] = (x, y)

        cv2.circle(clone, (x, y), 5, (0, 0, 255), -1)
        cv2.putText(clone, label, (x + 5, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
        index += 1
        cv2.imshow("Image", clone)

        if index == len(labels):
            h, w = clone.shape[:2]
            cx, cy = w // 2, h // 2

            for k in Xray_breaks:
                x, y = Xray_breaks[k]
                Xray_breaks[k] = (x - cx, cy - y)

            print("\n--> Centered Coordinates (origin at image center):")
            for key, val in Xray_breaks.items():
                print(f"{key} → {val}")

cv2.imshow("Image", clone)
cv2.setMouseCallback("Image", click_landmarks)
cv2.waitKey(0)
cv2.destroyAllWindows()


--> Centered Coordinates (origin at image center):
ulna break → (56, -30)
radius break → (-22, 69)


In [331]:
# Load YOLO Model 

model = YOLO(r"C:\Hackathon\3D bone mapping\working_code\best.pt")
results = model(img)

results[0].show()



0: 640x256 2 breaks, 142.2ms
Speed: 4.5ms preprocess, 142.2ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 256)


In [332]:
# Draw Bone Line in Image

h, w = img.shape[:2]
cx, cy = w // 2, h // 2

ulna_head     = (int(Xray_landmark['ulna head'][0] + cx), int(cy - Xray_landmark['ulna head'][1]))
ulna_tail     = (int(Xray_landmark['ulna tail'][0] + cx), int(cy - Xray_landmark['ulna tail'][1]))
radius_head   = (int(Xray_landmark['radius head'][0] + cx), int(cy - Xray_landmark['radius head'][1]))
radius_tail   = (int(Xray_landmark['radius tail'][0] + cx), int(cy - Xray_landmark['radius tail'][1]))

cv2.line(img, ulna_head, ulna_tail, (0, 0, 200), 5)
cv2.line(img, radius_head, radius_tail, (0, 200, 0), 5)

cv2.imshow("Lines", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [353]:
import math

def angle_from_negative_x(p1, p2, center=(0, 0)):

    # Shift points to origin (center)
    x1 = p1[0] - center[0]
    y1 = p1[1] - center[1]
    x2 = p2[0] - center[0]
    y2 = p2[1] - center[1]
    
    # Direction vector (p1 → p2)
    dx = x2 - x1
    dy = y2 - y1
    
    # Angle from +X-axis (standard atan2)
    angle_rad = math.atan2(dy, dx)
    angle_deg = math.degrees(angle_rad)
    
    # Convert to 0°-360° range (measured from +X)
    angle_deg = angle_deg % 360
    
    # Adjust to measure from -X-axis (clockwise)
    angle_from_neg_x = (angle_deg - 180) % 360

    
    if angle_from_neg_x <= 90:
        return -(90 - angle_from_neg_x)
    
    elif 90 <= angle_from_neg_x < 180:
        return   (angle_from_neg_x - 90)
    
    elif 180 <= angle_from_neg_x < 270:
        return (270 - angle_from_neg_x)
    
    elif angle_from_neg_x > 360:
        return (360 - angle_from_neg_x)


In [354]:
def get_split_ratio(point_top, point_bottom, split_point):
    
    y_top = point_top[1]
    y_bottom = point_bottom[1]
    y_split = split_point[1]

    if y_top < y_bottom:
        y_top, y_bottom = y_bottom, y_top

    if y_split == 0:
        y_split = 1e-5

    total_height = y_top + y_bottom
    split_ratio = total_height / y_split

    return split_ratio


In [355]:
def create_angle_mesh(mesh, angles, split_ratio):

    vertices = np.asarray(mesh.vertices)
    triangles = np.asarray(mesh.triangles)

    mid_y = (vertices[:, 1].min() + vertices[:, 1].max()) / split_ratio

    top_mask = vertices[:, 1] >= mid_y
    top_indices = np.where(top_mask)[0]

    index_map_top = -np.ones(len(vertices), dtype=int)
    index_map_top[top_indices] = np.arange(len(top_indices))

    top_triangle_mask = np.all(top_mask[triangles], axis=1)
    top_triangles = triangles[top_triangle_mask]
    mapped_top_triangles = index_map_top[top_triangles]

    top_vertices = np.copy(vertices[top_indices])

    angle_rad_top = np.radians(angles[0])
    R_top = mesh.get_rotation_matrix_from_axis_angle([0, 0, angle_rad_top])

    center_top = np.array([
        top_vertices[:, 0].mean(),
        mid_y,
        top_vertices[:, 2].mean()
    ])

    rotated_top_vertices = (R_top @ (top_vertices - center_top).T).T + center_top

    rotated_top_mesh = o3d.geometry.TriangleMesh()
    rotated_top_mesh.vertices = o3d.utility.Vector3dVector(rotated_top_vertices)
    rotated_top_mesh.triangles = o3d.utility.Vector3iVector(mapped_top_triangles)
    rotated_top_mesh.compute_vertex_normals()

    bottom_mask = vertices[:, 1] < mid_y
    bottom_indices = np.where(bottom_mask)[0]

    index_map_bottom = -np.ones(len(vertices), dtype=int)
    index_map_bottom[bottom_indices] = np.arange(len(bottom_indices))

    bottom_triangle_mask = np.all(bottom_mask[triangles], axis=1)
    bottom_triangles = triangles[bottom_triangle_mask]
    mapped_bottom_triangles = index_map_bottom[bottom_triangles]

    bottom_vertices = np.copy(vertices[bottom_indices])

    angle_rad_bottom = -1 * np.radians(angles[1])
    R_bottom = mesh.get_rotation_matrix_from_axis_angle([0, 0, angle_rad_bottom])

    center_bottom = np.array([
        bottom_vertices[:, 0].mean(),
        mid_y,
        bottom_vertices[:, 2].mean()
    ])

    rotated_bottom_vertices = (R_bottom @ (bottom_vertices - center_bottom).T).T + center_bottom

    rotated_bottom_mesh = o3d.geometry.TriangleMesh()
    rotated_bottom_mesh.vertices = o3d.utility.Vector3dVector(rotated_bottom_vertices)
    rotated_bottom_mesh.triangles = o3d.utility.Vector3iVector(mapped_bottom_triangles)
    rotated_bottom_mesh.compute_vertex_normals()

    final_mesh = rotated_top_mesh + rotated_bottom_mesh
    
    return final_mesh

In [356]:
ulna_angles = []

ulna_angles.append(angle_from_negative_x(Xray_landmark['ulna head'], Xray_breaks['ulna break']))
ulna_angles.append(angle_from_negative_x(Xray_landmark['ulna tail'], Xray_breaks['ulna break']))

In [357]:
print(ulna_angles)
print(get_split_ratio(Xray_landmark['ulna head'], Xray_landmark['ulna tail'], Xray_breaks['ulna break']))

[9.535465716606268, 6.966692522330845]
2.6666666666666665


In [358]:
radius_angles = []

radius_angles.append(angle_from_negative_x(Xray_landmark['radius head'], Xray_breaks['radius break']))
radius_angles.append(angle_from_negative_x(Xray_landmark['radius tail'], Xray_breaks['radius break']))

In [359]:
print(radius_angles)
print(get_split_ratio(Xray_landmark['radius head'], Xray_landmark['radius tail'], Xray_breaks['radius break']))

[11.663294034863839, 4.021990177020086]
-2.260869565217391


In [360]:
break_ulna = create_angle_mesh(mesh_ulna, ulna_angles, get_split_ratio(Xray_landmark['ulna head'], Xray_landmark['ulna tail'], Xray_breaks['ulna break']))

In [361]:
break_radius = create_angle_mesh(mesh_radius, radius_angles, get_split_ratio(Xray_landmark['radius head'], Xray_landmark['radius tail'], Xray_breaks['radius break']))

In [362]:
o3d.visualization.draw_geometries([break_radius, break_ulna])